# Data Cleaning

Load metadata and GO data, clean, and merge together

```
Dustin Michels
November 2017
```

In [1]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
from pandas.plotting import parallel_coordinates

In [2]:
data_path = '../../data/'
go_data_path = data_path + 'go_downloads/'
tax_data_path = data_path + 'taxonomy'
img_path = '../imgs/'

## Define Functions for Reading Data

### Get / Clean Meta Data

In [3]:
def get_meta_df():
    
    # Get and clean project meta data
    meta_df = pd.read_csv(data_path + "project_metadata_functional.csv")

    # Make column names neater
    meta_df.columns = meta_df.columns.str.strip()
    meta_df.columns = meta_df.columns.str.lower()
    meta_df.columns = meta_df.columns.str.replace(' ', '_')
    meta_df.rename(columns={'sample_details':'zone'}, inplace=True)

    # Split lat and long into seperate columns
    meta_df['lat'] = meta_df['lat/long'].str.split(',', 1).str[0]
    meta_df['long'] = meta_df['lat/long'].str.split(',', 1).str[1]

    # Clean up 'region' and 'zone' columns
    meta_df['region'] = meta_df['region'].str.strip()
    meta_df['zone'] = meta_df['zone'].str.strip()

    # Indicate categorical data
    meta_df['region'] = meta_df['region'].astype('category')
    meta_df['zone'] = meta_df['zone'].astype('category')
    meta_df['run_id'] = meta_df['run_id'].astype('category')

    # Drop a few categories
    meta_df.drop(
        ['downloaded','link_to_info', 'student', 'lat/long'],
        axis=1, inplace=True)
    
    return meta_df

### Get / Clean GO Data

In [4]:
def get_df_helper(idx):
    """Helper function for parsing GO CSVs"""
    
    filenames = meta_df['filename']
    names = ['go_id', 'name', 'namespace', 'read_count']

    # Read GO csv (for given index)
    filepath = f"{go_data_path}{filenames[idx]}"
    df = pd.read_csv(
        filepath, header=None, names=names)
    
    # Add run_id column
    df.insert(0, 'run_id', meta_df['run_id'][idx])
    
    # Sort by read_count
    df.sort_values('read_count', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Add read_percent column, based on read_count
    read_sum = df['read_count'].sum()
    df['read_percent'] = (df['read_count']/read_sum)
    
    # Drop some columns
    df.drop(['go_id','read_count'], axis=1, inplace=True)
    
    return df


def get_GO_df():
    """Call helper function with all GO annotations,
    concatenating resulting dataframes together
    """
    
    df = get_df_helper(0)
    for i in range(1, len(meta_df)):
        new_df = get_df_helper(i)
        df = pd.concat([df, new_df])
    df.reset_index(drop=True, inplace=True)

    # Indicate categorical data
    df['run_id'] = df['run_id'].astype('category')
    df['namespace'] = df['namespace'].astype('category')
    
    return df

### Get / Clean Taxonomy Data

In [46]:
def get_tax_df():
    """Read tax_summary.csv and into pandas
    and clean data into tiday format"""
    
    mapping_dict = {
        '00_SrnOcean_DCM':'ERR599104',
        '01_SrnOcean_SURF':'ERR599090',
        '02_SrnOcean_MESO':'ERR599008',
        '03_SPac_DCM':'ERR598948',
        '04_SPac_SURF':'ERR598992',
        '05_SPac_MESO':'ERR598999',
        '06_NPac_DCM':'ERR598995',
        '07_NPac_MESO':'ERR598980',
        '08_NPac_SURF':'ERR599142',
        '09_NAtl_SURF':'ERR599078',
        '10_AraSea_MESO':'ERR599031'
    }
    samples = list(mapping_dict.keys())
    run_id = list(mapping_dict.values())

    tax_df = pd.read_csv(f"{tax_data_path}/tax_summary.csv")
    tax_df = tax_df[tax_df['taxlevel']==2]

    tax_df.drop(
        ['taxlevel', 'daughterlevels', 'rankID','total'],
        axis=1, inplace=True)
    
    # Convert counts to percents
    for samp in samples:
        tax_df[samp] = tax_df[samp] / tax_df[samp].sum()

    # Fix names
    cols = ['taxon'] + run_id
    tax_df.columns = cols
    
    # Restructure
    tax_df = tax_df.melt(
        id_vars='taxon', var_name='run_id',
        value_name='tax_percent')
    
    # Sort columns
    tax_df = tax_df.sort_index(axis=1)
    
    return tax_df

## Merge Data Together

In [47]:
def merge_func(meta_df, go_df):
    """Merge go_df, meta_df together, using run_id"""
    
    # Merge meta/GO
    full_df = go_df.merge(meta_df, on='run_id')
    full_df.drop('filename', axis=1, inplace=True)
    
    return full_df

In [48]:
def merge_tax(meta_df, tax_df):
    """Merge tax_df, meta_df together, using run_id"""
    
    full_df = tax_df.merge(meta_df, on='run_id')
    full_df.drop('filename', axis=1, inplace=True)
    
    return full_df

## Make Top-25 dataframes

In [49]:
def top_25_func(full_df):
    """Get superset of 25 most abundant functional groups
    shared by all samples. This becomes 29 functional groups.
    Also: truncates func group names at 35 characters."""

    grouped_df = full_df.groupby('run_id').head(n=25)
    names = grouped_df.name.unique()

    # Select entries in full_df where name is one of the 29
    top_df = full_df[full_df['name'].isin(names)]

    # Make a deep copy (instead of using a slice of full_df)
    top_df = top_df.copy(deep=True)
    
    # Truncate function names at 35 characters
    top_df['name'] = top_df['name'].apply(
    lambda x: (x[:35] + '...') if len(x) > 38 else x)
    
    return top_df

In [50]:
def top_25_tax(full_df):
    """Get superset of 25 most abundant functional groups
    shared by all samples. This becomes 29 functional groups.
    Also: truncates func group names at 35 characters."""

    # Grab 25 largest, by taxonomy percent, for each group
    s = full_df.groupby('run_id')['tax_percent'].nlargest(25)
    
    # Query full dataframe for matching indicies
    top_df = full_df.iloc[s.index.droplevel(0)]
    
    # Make a deep copy (instead of using a slice of full_df)
    top_df = top_df.copy(deep=True)
    
    return top_df

## Run the Code!

In [51]:
meta_df = get_meta_df()
go_df = get_GO_df()
tax_df = get_tax_df()

full_go = merge_func(meta_df, go_df)
full_tax = merge_tax(meta_df, tax_df)

top_go = top_25_func(full_go)
top_tax = top_25_tax(full_tax)